# Import Libraries

In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialization site

In [81]:
standings_url = "https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats"

In [82]:
data = requests.get(standings_url)

In [83]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [84]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [85]:
data = requests.get(team_urls[1])

# First main Table Scores & Fixtures 1

In [86]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [87]:
matches.shape

(56, 19)

# Second Table/Adding attributes to first one 2

Table shooting need to merge attributes like:
- date
- Sh
- Dist
- FK
- PK
- PKatt

Need to think aboud expected column

In [88]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [89]:
data = requests.get(f"https://fbref.com{links[0]}")

In [90]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [91]:
shooting.shape

(57, 26)

In [92]:
shooting.columns = shooting.columns.droplevel()

In [93]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [94]:
team_data.shape

(56, 25)

# Third Table/Adding attributes to team_data one 3

Table Goalkeeping need to merge attributes like:
- date
- SoTA
- PKA
- PKsv
- PKm

Need to think about expected column

In [95]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/keeper' in l]

In [96]:
data = requests.get(f"https://fbref.com{links[0]}")

In [97]:
keeper = pd.read_html(data.text, match="Goalkeeping")[0]

In [98]:
keeper.shape

(57, 37)

In [99]:
keeper.columns = keeper.columns.droplevel()

In [100]:
#list(keeper.columns)

In [101]:
team_data = team_data.merge(keeper[["Date", "SoTA", "PKA", "PKsv", "PKm"]], on="Date")

In [102]:
team_data.shape

(56, 29)

# Next table Passing 4

Table Passing need to merge attributes like:
- date
- Cmp_Total
- Att_Total
- TotDist
- PrgDist

Need to think about expected column

In [103]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/passing' in l]

In [104]:
data = requests.get(f"https://fbref.com{links[0]}")

In [105]:
passing = pd.read_html(data.text, match="Passing")[0]

In [106]:
passing.shape

(57, 33)

In [107]:
passing.columns = passing.columns.droplevel()

In [108]:
passing.shape

(57, 33)

In [109]:
passing.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist', 'Cmp', 'Att',
       'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Ast', 'xAG', 'xA',
       'KP', '1/3', 'PPA', 'CrsPA', 'Prog', 'Match Report'],
      dtype='object')

In [110]:
passing.columns.values[10] = "Cmp_Total"
passing.columns.values[11] = "Att_Total"

In [111]:
passing.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Cmp_Total', 'Att_Total', 'Cmp%', 'TotDist', 'PrgDist',
       'Cmp', 'Att', 'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Cmp', 'Att', 'Cmp%', 'Ast',
       'xAG', 'xA', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog', 'Match Report'],
      dtype='object')

In [112]:
team_data = team_data.merge(passing[["Date", "Cmp_Total", "Att_Total", "TotDist", "PrgDist"]], on="Date")

In [113]:
team_data.shape

(56, 33)

In [114]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,PK,PKatt,SoTA,PKA,PKsv,PKm,Cmp_Total,Att_Total,TotDist,PrgDist
0,2017-08-08,20:45,Super Cup,UEFA Super Cup,Tue,Neutral,L,1,2,es Real Madrid,...,0,0,6,0,0,0,NaN,NaN,NaN,NaN
1,2017-08-13,16:00,Premier League,Matchweek 1,Sun,Home,W,4,0,West Ham,...,0,0,1,0,0,0,442.0,538.0,7647.0,2869.0
2,2017-08-19,12:30,Premier League,Matchweek 2,Sat,Away,W,4,0,Swansea City,...,0,0,0,0,0,0,542.0,637.0,8245.0,2715.0
3,2017-08-26,17:30,Premier League,Matchweek 3,Sat,Home,W,2,0,Leicester City,...,0,1,4,0,0,0,604.0,724.0,9545.0,3253.0
4,2017-09-09,17:30,Premier League,Matchweek 4,Sat,Away,D,2,2,Stoke City,...,0,0,5,0,0,0,525.0,628.0,9195.0,2905.0


# Next table Pass Types 5

Table Passi Types to merge attributes like:
- TI
- CK

Need to think about expected column

In [115]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/passing_types' in l]

In [116]:
data = requests.get(f"https://fbref.com{links[0]}")

In [117]:
pass_types = pd.read_html(data.text, match="Pass Types")[0]

In [118]:
pass_types.shape

(57, 26)

In [119]:
pass_types.columns = pass_types.columns.droplevel()

In [120]:
pass_types.shape

(57, 26)

In [121]:
team_data = team_data.merge(pass_types[["Date", "TI", "CK"]], on="Date")

In [122]:
team_data.shape

(56, 35)

# Next table Goal and Shot Creation 6

Table Goal and Shot need to merge attributes like:
- SCA
- GCA

Need to think about expected column

In [123]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/gca' in l]

In [124]:
data = requests.get(f"https://fbref.com{links[0]}")

In [125]:
gca = pd.read_html(data.text, match="Goal and Shot Creation")[0]

In [126]:
gca.shape

(57, 25)

In [127]:
gca.columns = gca.columns.droplevel()

In [128]:
gca.shape

(57, 25)

In [129]:
team_data = team_data.merge(gca[["Date", "SCA", "GCA"]], on="Date")

In [130]:
team_data.shape

(56, 37)

# Next table Defensive Actions 7

Table Defensive need to merge attributes like:
- Tkl_Defensive
- TklW_Defensive
- Blocks_Defensive

Need to think about expected column

In [131]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/defense' in l]

In [132]:
data = requests.get(f"https://fbref.com{links[0]}")

In [133]:
defension_actions = pd.read_html(data.text, match="Defensive Actions")[0]

In [134]:
defension_actions.shape

(57, 27)

In [135]:
defension_actions.columns = defension_actions.columns.droplevel()

In [136]:
defension_actions.shape

(57, 27)

In [137]:
defension_actions.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Tkl', 'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Tkl',
       'Att', 'Tkl%', 'Past', 'Blocks', 'Sh', 'Pass', 'Int', 'Tkl+Int', 'Clr',
       'Err', 'Match Report'],
      dtype='object')

In [138]:
defension_actions.columns.values[10] = "Tkl_Defensive"
defension_actions.columns.values[11] = "TklW_Defensive"
defension_actions.columns.values[19] = "Blocks_Defensive"

In [139]:
defension_actions.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Tkl_Defensive', 'TklW_Defensive', 'Def 3rd', 'Mid 3rd',
       'Att 3rd', 'Tkl', 'Att', 'Tkl%', 'Past', 'Blocks_Defensive', 'Sh',
       'Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Match Report'],
      dtype='object')

In [140]:
team_data = team_data.merge(defension_actions[["Date", "Tkl_Defensive", "TklW_Defensive", "Blocks_Defensive"]], on="Date")
team_data.shape

(56, 40)

# Next table Possesion 8

Table Possesion need to merge attributes like:
- date
- Touches
- Def Pen
- Def 3rd
- Mid 3rd
- Att 3rd
- Dribbles_Succ
- Dribbles_Att
- Rec

Need to think about expected column

In [141]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/possession' in l]

In [142]:
data = requests.get(f"https://fbref.com{links[0]}")

In [143]:
possession = pd.read_html(data.text, match="Possession")[0]

In [144]:
possession.shape

(57, 26)

In [145]:
possession.columns = possession.columns.droplevel()

In [146]:
possession.shape

(57, 26)

In [147]:
possession.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Poss', 'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd',
       'Att 3rd', 'Att Pen', 'Live', 'Succ', 'Att', 'Succ%', 'Mis', 'Dis',
       'Rec', 'Prog', 'Match Report'],
      dtype='object')

In [148]:
possession.columns.values[18] = "Dribbles_Succ"
possession.columns.values[19] = "Dribles_Att"

In [149]:
possession.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Poss', 'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd',
       'Att 3rd', 'Att Pen', 'Live', 'Dribbles_Succ', 'Dribles_Att', 'Succ%',
       'Mis', 'Dis', 'Rec', 'Prog', 'Match Report'],
      dtype='object')

In [150]:
team_data = team_data.merge(possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Dribbles_Succ", "Dribles_Att", "Rec"]], on="Date")
team_data.shape

(56, 48)

# Next table Miscellaneous stats 9

Table Possesion need to merge attributes like:
- date
- CrdY
- CrdR
- 2CrdY
- Fls
- Fld
- Off
- PKwon
- PKcon
- OG

Need to think about expected column

In [151]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/misc' in l]

In [152]:
data = requests.get(f"https://fbref.com{links[0]}")

In [153]:
msc = pd.read_html(data.text, match="Miscellaneous Stats")[0]

In [154]:
msc.shape

(57, 27)

In [155]:
msc.columns = msc.columns.droplevel()

In [156]:
msc.shape

(57, 27)

In [157]:
team_data = team_data.merge(msc[["Date", "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "PKwon", "PKcon", "OG"]], on="Date")
team_data.shape

(56, 57)

# Convert to CSV file

In [158]:
team_data.to_csv('EPL_2021.csv', mode='a', index=False, header=False)